#run this way of training
#try to penalize lack of smoothness somehow
#tru KL div as cost function (remove the *256 when creatinng histograms y)

In [170]:
!pip install fast-histogram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch
import torch.nn as nn
import numpy as np
import torchvision as T
import tqdm
import matplotlib.pyplot as plt

from fast_histogram import histogram1d

In [171]:
nb_images = 5000
batch_size = 25
img_size = 256*256

In [172]:
def torch_gaussian_image(mean,variance):
  img = (torch.randn(256*256)*variance)+mean
  #img += torch.rand(256*256)*(torch.numel(img)/10)
  img = img.reshape((1,256,256))
  return img

In [173]:
means = 50
variance = 30
minibatches_X = [torch.stack([torch_gaussian_image(np.random.randint(0,256),np.random.randint(2,60)) for i in range(batch_size)]) for l in range(0,nb_images,batch_size)]
minibatches_y = [torch.stack([256.*torch.Tensor(histogram1d(image.flatten(),bins=256,range=[0,256]))/img_size for image in batch]) for batch in minibatches_X]

minibatches_X = [batch.to("cuda:0") for batch in minibatches_X]
minibatches_y = [batch.to("cuda:0") for batch in minibatches_y]

In [174]:
minibatches_X[0].shape

torch.Size([25, 1, 256, 256])

In [175]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 =  nn.Conv2d(1,10,3,2)
    self.b1 = nn.BatchNorm2d(10)
    
    self.conv2 =  nn.Conv2d(10,10,3,2)
    self.b2 = nn.BatchNorm2d(10)

    self.conv3 =  nn.Conv2d(10,10,3,2)
    self.b3 = nn.BatchNorm2d(10)

    self.fl = nn.Flatten()
    self.lin = nn.Linear(31*31*10,256)
    self.b11 = nn.BatchNorm1d(256)

  def forward(self,x):
    x = self.conv1(x)
    x = self.b1(x)
    x = nn.functional.relu(x)

    x = self.conv2(x)
    x = self.b2(x)
    x = nn.functional.relu(x)

    x = self.conv3(x)
    x = self.b3(x)


    x = nn.functional.relu(x)


    x = self.fl(x)


    x = self.lin(x)
    x = nn.ReLU()(self.b11(x))
    return x#*20

In [176]:
def create_the_model():
  model = Model().to("cuda:0")
  loss = nn.MSELoss()
  opt = torch.optim.Adam(model.parameters(),lr=0.001)
  return model,loss,opt

In [177]:
model,loss,opt = create_the_model()

model(minibatches_X[0]).shape

torch.Size([25, 256])

In [178]:
def plot_comparisons(model,pred,histograms):
  plt.figure(figsize=(len(pred)*10,10))
  for i in range(len(pred)):
    plt.subplot(2,len(pred),i+1)
    plt.plot(pred[i].cpu().detach())
    plt.title("prediction")
    plt.subplot(2,len(pred),len(pred)+1+i)
    plt.plot(histograms[i].cpu().detach())
    plt.title("real")
  plt.show()

In [179]:
def train_the_model(nb_epochs):
  losses = []

  lrs = list(opt.param_groups[0]['lr']*torch.exp(torch.linspace(10,1,nb_epochs))/np.exp(10) +opt.param_groups[0]['lr']/20.)
  i=0
  while i<nb_epochs:# or np.mean(losses[-300:])>losses[-1]:#stop only when loss didn't improve overall in the last 10 iterations
    i+=1
    for (X,y) in zip(minibatches_X,minibatches_y):


      pred = model(X)
      l = loss(pred,y)
      
      if i<nb_epochs:
        opt.param_groups[0]['lr'] = lrs[i]
      else:
        pass
        lrs.append(opt.param_groups[0]['lr'])

      opt.zero_grad()
      l.backward()
      opt.step()
      losses.append(l.item())
    print("\nepoch",i,"loss improved by",np.mean(losses[-300:])-losses[-1])
  plot_comparisons(model,pred[:5],y[:5])
  plt.title("learning rates (up to 100 first epochs)")
  plt.plot(lrs[:100])
  plt.show()
  return losses

In [180]:
model,loss,opt = create_the_model()

losses = train_the_model(300)
plt.plot(np.log(losses))
plt.title("losses (log scale)")


epoch 1 loss improved by 0.05657244086265578

epoch 2 loss improved by 0.00754909704128881

epoch 3 loss improved by -0.002835822701454216

epoch 4 loss improved by 0.04529867529869058

epoch 5 loss improved by 0.06193053106466939

epoch 6 loss improved by 0.08542652706305187

epoch 7 loss improved by 0.0847077029943466

epoch 8 loss improved by 0.099396835565567

epoch 9 loss improved by 0.04929935922225326

epoch 10 loss improved by 0.03510681539773941

epoch 11 loss improved by 0.043281364639600195

epoch 12 loss improved by 0.05182455271482467

epoch 13 loss improved by 0.06108018219470979

epoch 14 loss improved by 0.04150905976692831

epoch 15 loss improved by 0.026473870327075266

epoch 16 loss improved by 0.01726080040136968

epoch 17 loss improved by 0.0053782347838085

epoch 18 loss improved by -0.002174497644106488

epoch 19 loss improved by -0.0027686471740404617

epoch 20 loss improved by -0.00440079018473627

epoch 21 loss improved by -0.030029671986897766

epoch 22 loss

KeyboardInterrupt: ignored

In [ ]:
print(losses[-5:])